In [1]:
import numpy as np
import pandas as pd
import time
from datetime import datetime
import json
%load_ext autoreload
%autoreload 2
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))
from tqdm.notebook import tqdm
tqdm.pandas()
import requests
import os
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from pathlib import Path
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

/var/folders/zr/0mklm_6s5zg8kfjw3xy0w3bh0000gn/T/ipykernel_31653/3700585894.py:8: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [2]:
TOP_DIR='/Users/tylermartin/Documents/programming/london_weather'

In [8]:
def get_unix_time():
    now = datetime.now()
    unix = int(time.mktime(now.timetuple()))
    return unix


def dt_to_unix(dt):
    return int(time.mktime(dt.timetuple()))


def response_to_df(response):
    day = json.loads(response.text)
    df = pd.DataFrame(day['hourly']['data'])
    to_drop = 'dewPoint ozone precipType pressure uvIndex'.split()
    to_drop = [c for c in to_drop if c in df.columns]
    df.drop(to_drop, axis=1, inplace=True)
    return df


def load_year(place):
    path = f'data/{place}/*csv'
    files = glob(path)
    logging.info(f'{len(files)} days downloaded')
    df = pd.concat([pd.read_csv(f) for f in files], sort=False)
    df.reset_index(inplace=True, drop=True)
    df.sort_values('time', inplace=True)
    df['dt'] = df.time.apply(unix_to_ts)
    return df

In [25]:
key=read_key()

In [26]:
lat,long=read_latlong()['london']

In [59]:
files =list(Path(TOP_DIR).glob('data/*.parquet'))
files.sort()

In [60]:
df=pd.concat([pd.read_parquet(f) for f in tqdm(files)])

  0%|          | 0/10 [00:00<?, ?it/s]

In [61]:
df

,unix,temp,apparent_temp,humidity,wind_speed,wind_bearing,cloud_cover,visibility,ts
0,1546300800,47.88,43.79,0.77,8.97,288.0,0.99,6.216,2019-01-01 00:00:00
1,1546304400,47.51,43.15,0.78,9.46,291.0,0.99,6.216,2019-01-01 01:00:00
2,1546308000,48.50,44.48,0.75,9.17,296.0,0.99,6.216,2019-01-01 02:00:00
3,1546311600,48.50,44.38,0.73,9.45,290.0,0.99,6.216,2019-01-01 03:00:00
4,1546315200,48.24,44.23,0.72,8.96,292.0,0.96,6.216,2019-01-01 04:00:00
...,...,...,...,...,...,...,...,...,...
19,1547146800,40.80,37.34,0.88,5.10,280.0,0.97,4.479,2019-01-10 19:00:00
20,1547150400,41.19,37.45,0.89,5.58,276.0,0.96,3.683,2019-01-10 20:00:00
21,1547154000,41.19,37.37,0.91,5.70,262.0,0.89,3.500,2019-01-10 21:00:00
22,1547157600,41.95,37.76,0.91,6.53,276.0,0.74,3.161,2019-01-10 22:00:00
